<a href="https://colab.research.google.com/github/Johar98/Machine-Learning-Algorithm/blob/main/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import pandas as pd

In [16]:
CSV_COLUMN_NAME = ['SepalLength','SepalWidth','PetalLength','PetalWidth','Species'] #data set headers for the columns / properties
SPECIES = ['Setosa','Versicolor','Virginica'] # plant name

In [17]:
train_path = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
#for training our model
test_path = tf.keras.utils.get_file("iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")
#for testing
#here we use keras(a module inside of tensorflow) to grab our datasets and read them into a pandas dataframe

In [18]:
train = pd.read_csv(train_path, names=CSV_COLUMN_NAME, header=0) #load train
#row 0 is header
test = pd.read_csv(test_path, names=CSV_COLUMN_NAME, header=0) #load test
print(train.head())

   SepalLength  SepalWidth  PetalLength  PetalWidth  Species
0          6.4         2.8          5.6         2.2        2
1          5.0         2.3          3.3         1.0        1
2          4.9         2.5          4.5         1.7        2
3          4.9         3.1          1.5         0.1        0
4          5.7         3.8          1.7         0.3        0


In [19]:
train_y = train.pop('Species')
test_y = test.pop('Species')
print(train.head())

   SepalLength  SepalWidth  PetalLength  PetalWidth
0          6.4         2.8          5.6         2.2
1          5.0         2.3          3.3         1.0
2          4.9         2.5          4.5         1.7
3          4.9         3.1          1.5         0.1
4          5.7         3.8          1.7         0.3


In [20]:
def input_fn(features, labels,training=True, batch_size=256):
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
  if training: #shuffle and repeat if you are in training mode
    dataset = dataset.shuffle(1000).repeat()
  return dataset.batch(batch_size)

In [21]:
my_feature_columns = []
for key in train.keys():# give us all the columns
  my_feature_columns.append(tf.feature_column.numeric_column(key=key)) #current key = random key
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [22]:
classifier = tf.estimator.DNNClassifier( #.estimator = store/build pre made model from tf
    feature_columns=my_feature_columns, #two hidden layers of 30 and 10 nodes respectively use to build an artichecter of neural network
    hidden_units=[30,10], # 30 = 1st node , 10 = 2nd node
    n_classes= 3) # the model must choose between 3 classes

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpu1nd1546', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [23]:
classifier.train(
    input_fn=lambda:input_fn(train, train_y, training = True),
     steps=5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpu1nd1546/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.8089938, step = 0
INFO:tensorflow:global_step/sec: 482.192
INFO:tensorflow:loss = 1.3152995, step = 100 (0.212 sec)
INFO:tensorflow:global_step/sec: 631.667
INFO:tensorflow:loss = 1.1653509, step = 200 (0.155 sec)
INFO:tensorflow:global_step/sec: 620.957
INFO:tensorflow:loss = 1.0516269, step = 300 (0.163 sec)
INFO:tensorflow:global_step/sec: 629.274
INFO:tensorflow:loss = 0.97598577, step = 400 (0.159 sec)
INFO:tensorflow:global_step/sec: 589.539
INFO:tensorflow:loss = 0.953057, step = 500 (0.170 sec)
INFO:t

In [24]:
eval_result = classifier.evaluate(
    input_fn=lambda:input_fn(test,test_y,training=False))
print('\n Test set accuracy:{accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-03-20T17:25:33Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpu1nd1546/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.20449s
INFO:tensorflow:Finished evaluation at 2021-03-20-17:25:33
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.53333336, average_loss = 0.7090868, global_step = 5000, loss = 0.7090868
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmpu1nd1546/model.ckpt-5000

 Test set accuracy:0.533



In [26]:
def input_fn(features, batch_size=256): #convert the input to a dataset without labels
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)
features= ['SepalLength','SepalWidth','PetalLength','PetalWidth'] #these are the feature
predict = {}
print("Please type numeric values as prompted")
for feature in features:# for each feature we are going to wait for valid respond once we get some valid response 
  valid = True          # then we will add that to dictonary
  while valid :
    val = input(feature + ":")
    if not val.isdigit():valid=False
  predict[feature]=[float(val)] # list of value what we have // also show input function to put data of leaf
predictions = classifier.predict(input_fn=lambda:input_fn(predict))
for pred_dict in predictions:
  print(pred_dict)
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]
  print('prediction is "{}" ({:.1f}'.format(
      SPECIES[class_id],100 * probability))

Please type numeric values as prompted
SepalLength:1.2
SepalWidth:2.1
PetalLength:2.1
PetalWidth:1.1
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpu1nd1546/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([-0.65726596, -0.24471274, -0.11067313], dtype=float32), 'probabilities': array([0.23595884, 0.35645592, 0.4075853 ], dtype=float32), 'class_ids': array([2]), 'classes': array([b'2'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
prediction is "Virginica" (40.8
